# Demand Forecasting Using LSTM (Synthetic Data)

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [ ]:

np.random.seed(42)
months = np.arange(1, 97)
demand = 100 + 10 * np.sin(2 * np.pi * months / 12) + np.random.normal(0, 5, 96)
df = pd.DataFrame({'Month': pd.date_range(start='2016-01-01', periods=96, freq='M'), 'Demand': demand})
df.set_index('Month', inplace=True)
df.plot(title="Synthetic Monthly Demand Data", figsize=(10, 4))
plt.show()


In [ ]:

scaler = MinMaxScaler()
scaled_demand = scaler.fit_transform(df[['Demand']])

def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)

SEQ_LENGTH = 12
X, y = create_sequences(scaled_demand, SEQ_LENGTH)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:

model = Sequential([
    LSTM(50, activation='relu', input_shape=(SEQ_LENGTH, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, validation_split=0.1, verbose=1)


In [ ]:

y_pred = model.predict(X_test)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)
print("RMSE:", np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)))
print("MAE:", mean_absolute_error(y_test_inv, y_pred_inv))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.legend()
plt.title("Actual vs Predicted Demand")
plt.show()
